# Web Search Agent Implementation

1. [Introduction](#introduction)
2. [Setting Up the Environment](#setup)
3. [Implementing the Tools](#tools)
4. [Building the Web Search Agent](#building)
5. [Running the Agent](#running)
6. [Conclusion](#conclusion)


## 1. Introduction <a name="introduction"></a>

The Web Search Agent is an AI-powered tool that combines the capabilities of Large Language Models (LLMs) with web search functionality. It can understand user queries, perform web searches, parse webpage content, and generate informative responses.

Key components of our implementation include:
- [Mirascope](https://www.mirascope.io/) library for LLM interactions
- [OpenAI](https://openai.com/)'s GPT model for natural language processing
- Custom tools for web searching and webpage parsing


## 2. Setting Up the Environment <a name="setup"></a>

First, we need to install the required packages and import the necessary modules.

In [ ]:
!pip install "mirascope[openai]" requests beautifulsoup4 pydantic duckduckgo-search

In [ ]:
import os
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from pydantic import BaseModel

from mirascope.core import BaseMessageParam, openai, prompt_template

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## 3. Implementing the Tools <a name="tools"></a>

Now, let's implement our tools: `web_search` and `parse_webpage`. These functions will allow our agent to search the web and extract content from webpages.

In [ ]:
MAX_RESULTS = 10


def web_search(text: str) -> str:
    """Search the web for the given text.

    Args:
        text: The text to search for.

    Returns:
        The search results for the given text, containing the `title`, `href`, and `body`.
    """
    try:
        results = DDGS(proxy=None).text(text, max_results=MAX_RESULTS)
        return "\n\n".join(
            [
                "title: {title}\nhref: {href}\nbody: {body}".format(**result)
                for result in results
            ]
        )
    except Exception as e:
        return f"{type(e)}: Failed to search the web for text"


def parse_webpage(link: str) -> str:
    """Parse the paragraphs of the webpage found at `link`.

    Args:
        link: The URL of the webpage.

    Returns:
        The parsed paragraphs of the webpage, separated by newlines.
    """
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")
        return "\n".join([p.text for p in soup.find_all("p")])
    except Exception as e:
        return f"{type(e)}: Failed to parse content from URL"

## 4. Building the Web Search Agent <a name="building"></a>

Now that we have our tools ready, let's build the WebSearchAgent class. This class will use the Mirascope framework to create an AI agent that can interact with users, perform web searches, and provide informative responses.

In [ ]:
class WebSearchAgent(BaseModel):
    history: list[BaseMessageParam | openai.OpenAIMessageParam] = []

    @openai.call("gpt-4o-mini", stream=True, tools=[web_search, parse_webpage])
    @prompt_template("""
        SYSTEM:
        Your task is to answer a user's query. The current date is {current_date}.

        You have access to the following tools:
        - `web_search`: Search the web for information.
        - `parse_webpage`: Parse the content of a webpage.

        When calling the `web_search` tool, the `body` is simply the body of the search
        result. You MUST then call the `parse_webpage` tool to get the actual content
        of the webpage. It is up to you to determine which search results to parse.

        Once you have gathered all of the information you need, generate your response,
        which should strike the right balance between brevity and completeness. Your answer
        to the user's query should be concise yet comprehensive. When discussing recent
        events or developments, make sure to consider the current date provided.

        MESSAGES: {self.history}
        USER: {query}
        """)
    def _stream(self, query: str):
        return {
            "computed_fields": {"current_date": datetime.now().strftime("%Y-%m-%d")}
        }

    def _step(self, query: str):
        stream = self._stream(query)
        tools_and_outputs = []
        for chunk, tool in stream:
            if tool:
                tools_and_outputs.append((tool, tool.call()))
            else:
                print(chunk.content, end="", flush=True)
        if stream.user_message_param:
            self.history.append(stream.user_message_param)
        self.history.append(stream.message_param)
        if tools_and_outputs:
            self.history += stream.tool_message_params(tools_and_outputs)
            self._step("")

    def run(self):
        while True:
            query = input("(User): ")
            if query.lower() in ["exit", "quit"]:
                break
            print("(Assistant): ", end="", flush=True)
            self._step(query)
            print("")

Let's break down the key components of our WebSearchAgent class:

1. **@openai.call decorator**: This decorator configures the LLM call using OpenAI's model. It specifies the use of the "gpt-4o-mini" model, enables streaming, and provides access to the `parse_webpage` and `web_search` tools.

2. **@prompt_template decorator**: This decorator defines the prompt template sent to the LLM. It allows for dynamic insertion of variables like `{current_date}` and `{self.history}`.

3. **_stream method**: This method generates streaming responses using the OpenAI model. The actual implementation is provided by the Mirascope library, so we don't need to implement the function body.

4. **_step method**: This method processes a single user query. It handles the streaming response, tool calls, and updates the conversation history.

5. **run method**: This method provides a simple command-line interface for interacting with the agent. It continuously prompts for user input until the user chooses to exit.

## 5. Running the Agent <a name="running"></a>

Now that we have implemented our WebSearchAgent, let's create an instance and run it. Here's an example of how the agent responds to a query about upcoming Python releases:

In [ ]:
agent = WebSearchAgent()
agent.run()

# > (User): What is the next Python release version?
# > (Assistant): The next Python release will be **Python 3.13**, expected to be officially released on **October 1, 2024**. This version is currently in prerelease status. After this, Python 3.14 is scheduled for release on **October 1, 2025**, marking the beginning of a new feature branch.

# Python 3.13 is important as it will introduce new features and enhancements, while Python 3.14 is anticipated to follow in the established release cadence outlined in PEP 602, which involves a 17-month development period and periodic updates thereafter.

You can now interact with your Web Search Agent! Try asking it questions that require web searches, like "What are the latest developments in artificial intelligence?" or "Can you summarize the plot of the latest popular movie?"

Remember, to exit the agent, simply type "exit" or "quit".

## 6. Conclusion <a name="conclusion"></a>

You've successfully implemented a Web Search Agent using the Mirascope library. This agent demonstrates the power of combining LLMs with custom tools for web searching and content parsing.

Key takeaways:
1. Custom tools like `web_search` and `parse_webpage` extend the agent's functionality.
2. The `@openai.call` and `@prompt_template` decorators streamline the configuration of LLM interactions.
3. Streaming responses allow for real-time interaction with the agent.

This implementation serves as a starting point. You can further enhance the agent by:
- Implementing a more sophisticated web search function
- Adding error handling and retries for web requests
- Extending the agent's capabilities with additional tools
- Optimizing the prompt template for better responses
